In [ ]:
#hide
IN_COLAB = 'google.colab' in str(get_ipython())
if IN_COLAB:
  !pip3 install -Uqq fastbook

     |████████████████████████████████| 727kB 5.6MB/s 
     |████████████████████████████████| 194kB 15.0MB/s 
     |████████████████████████████████| 51kB 5.6MB/s 
     |████████████████████████████████| 1.1MB 13.9MB/s 
     |████████████████████████████████| 61kB 8.0MB/s 


In [ ]:
#hide
if IN_COLAB:
  from google.colab import drive
  drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
#hide
if IN_COLAB:
  from pathlib import Path
  from nbdev.imports import Config
  project_path = Path('/content/drive/My Drive/Colab Notebooks/github/sdt')
  get_ipython().magic(f'cd {project_path}')
  get_ipython().magic(f'cd {Config().nbs_path}')

/content/drive/My Drive/Colab Notebooks/github/sdt
/content/drive/My Drive/Colab Notebooks/github/sdt


In [ ]:
#hide
from fastai.vision.all import *

from sdt.data  import *
from sdt.model import *
from sdt.loss  import *

# SDT

This package is an implementation of the paper [Distilling a Neural Network Into a Soft Decision
Tree](https://arxiv.org/pdf/1711.09784.pdf) by Nicholas Frosst, Geoffrey Hinton.


**Abstract**
Deep neural networks have proved to be a very effective
way to perform classification tasks. But it is hard to explain why a learned network makes
a particular classification decision on a particular test case. This is
due to their reliance on distributed hierarchical representations. If we
could take the knowledge acquired by the neural net and express the
same knowledge in a model that relies on hierarchical decisions instead,
explaining a particular decision would be much easier. We describe a way
of using a trained neural net to create a type of soft decision tree that
generalizes better than one learned directly from the training data.

## Install

`pip install git+https://github.com/numb3r33/sdt.git`

## How to use

In [ ]:
#ignore
path = untar_data(URLs.MNIST)
dls  = get_dls(path, bs=128)

In [ ]:
#ignore
current_depth = 0
depth = 4

tree      = Tree(input_dim=784, output_dim=10)
n_leaves  = 2 ** depth
n_classes = 10
n_epochs  = 40

tree = sdt_init(current_depth , depth, tree, input_dim=784, output_dim=n_classes)

# SDT
sdt       = SDT(tree=tree, n_leaves=n_leaves, n_classes=n_classes)
criterion = SDTLoss(lambda_=0.05)

In [ ]:
#ignore
learn = Learner(dls, 
                sdt, 
                loss_func=criterion, 
                opt_func=partial(SGD, lr=.2, mom=0.9), 
                cbs=SDTCallback(tree, n_leaves), 
                metrics=partial(accuracy_multi, sigmoid=False))

In [ ]:
#ignore
learn.fit_one_cycle(n_epoch=n_epochs)

![training_loop](images/training_loop.png)

## Learned Filters

![learned_filters](images/learned_filters.png)